### 할로윈 전략
1. 11월 첫날에 매수 -> 4월 말일에 매도
2. 11월 첫날의 시가를 매수 가격
3. 4월 말일의 종가를 매도 가격
4. 수익율 : 4월 말일의 종가 / 11월 첫날의 시가
5. 누적 수익율 계산

In [5]:
from datetime import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

In [6]:
# 데이터 로드 
df = pd.read_csv("../../csv/AAPL.csv")

In [7]:
# 컬럼에 Date 존재하면 Date를 인덱스로 설정 
if 'Date' in df.columns:
    df.set_index('Date', inplace=True)

In [8]:
# index를 시계열 데이터로 변경 
df.index = pd.to_datetime(df.index)

In [9]:
# 2010년 11월 첫날의 시가의 데이터
df.loc['2010-11-01' : '2010-12-01', 'Open' ][0]

C:\Users\moons\AppData\Local\Temp\ipykernel_4884\1676377478.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc['2010-11-01' : '2010-12-01', 'Open' ][0]


43.174286

In [10]:
df.loc['2010-11-01': , 'Open'][0]

C:\Users\moons\AppData\Local\Temp\ipykernel_4884\1022133651.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc['2010-11-01': , 'Open'][0]


43.174286

In [11]:
df.loc['2010-11', 'Open'][0]

C:\Users\moons\AppData\Local\Temp\ipykernel_4884\1424043316.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc['2010-11', 'Open'][0]


43.174286

In [12]:
# 2011년 4월의 말일의 종가
df.loc['2011-04', 'Close'][-1]

C:\Users\moons\AppData\Local\Temp\ipykernel_4884\2631509459.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.loc['2011-04', 'Close'][-1]


50.01857

In [13]:
# 2000년부터 2010년까지 할로윈 전략으로 수익율 계산

price_df = pd.DataFrame()

for year in range(2000, 2010, 1):
    # 매수 year-11
    buy_mon = f"{year}-11"
    # 매도 (year+1)-04
    sell_mon = f"{year+1}-04"
    buy = df.loc[buy_mon, ].head(1)
    sell = df.loc[sell_mon, ].tail(1)

    price_df = pd.concat(
        [price_df, buy, sell], axis=0
    )

In [14]:
price_df = price_df[ ['Open', 'Close'] ]

In [15]:
# price_df.loc[ : , ['Open', 'Close'] ]

In [16]:
# 수익율 컬럼을 생성 1을 대입 
price_df['rtn'] = 1

In [17]:
price_df.head(1)

,Open,Close,rtn
Date,,,
2000-11-01,1.388393,1.464286,1


In [18]:
# 인덱스 위치의 값이 홀수인 경우 수익율 발생 
# for idx in range(len(price_df)):
#     if idx % 2 == 1:
        # 홀수인 경우
for idx in range(1, len(price_df), 2):
    sell = price_df.iloc[idx, 1]
    # buy = price_df.iloc[idx-1, 0]
    buy = price_df.shift().iloc[idx, 0]
    rtn = sell / buy
    price_df.iloc[idx, -1] = rtn

C:\Users\moons\AppData\Local\Temp\ipykernel_4884\2833613327.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.3113822959349406' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  price_df.iloc[idx, -1] = rtn


In [19]:
price_df['acc_rtn'] = price_df['rtn'].cumprod()

In [20]:
# 수익율 발생하는 조건 -> 4월 -> 시계열 데이터에서 월만 추출하면 확인 가능
price_df.index[1].strftime('%m') == '04'

True

In [21]:
price_df.index[1].month == 4

True

In [22]:
price_df['rtn2'] = 1
for idx in price_df.index:
    # idx의 월 데이터가 4인 경우
    # if idx.strftime('%m') == '04':
    if idx.month == 4:
        rtn = price_df.loc[idx, 'Close'] / \
            price_df.shift().loc[idx, 'Open']
        price_df.loc[idx, 'rtn2'] = rtn

C:\Users\moons\AppData\Local\Temp\ipykernel_4884\1991335432.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.3113822959349406' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  price_df.loc[idx, 'rtn2'] = rtn


In [23]:
price_df[['rtn', 'rtn2']]

,rtn,rtn2
Date,,
2000-11-01,1.000000,1.000000
2001-04-30,1.311382,1.311382
2001-11-01,1.000000,1.000000
2002-04-30,1.375071,1.375071
2002-11-01,1.000000,1.000000
2003-04-30,0.892095,0.892095
2003-11-03,1.000000,1.000000
2004-04-30,1.129216,1.129216
2004-11-01,1.000000,1.000000


In [24]:
# 투자의 시작시간  -> 년도 -> 문자를 시계열로 변환 -> 시계열에서 년도만 특정 월
start = '2000-01-01'
# 시계열 데이터로 변환 
t_s = pd.to_datetime(start)
# t_s의 년도를 추출 
t_s2 = t_s.replace(month=11)

In [25]:
# 시작일로 부터 5개월 뒤
t_s2 + relativedelta(months=5)

Timestamp('2001-04-01 00:00:00')

### 할로윈 투자 전략 함수화
- 매개변수 5개 
    - 데이터프레임 : 필수
    - 시작시간 : 2010-01-01 기본값
    - 종료시간 : 현재 시간 기본값
    - 기준이 되는 컬럼 : 'Adj Close' 기본값
    - 투자의 시작 월 : 11 기본값

In [26]:
def six_month(
        _df, 
        _start = '2010-01-01', 
        _end = datetime.now(), 
        _col = 'Adj Close', 
        _month = 11
):
    # 복사본 생성 
    df = _df.copy()
    # Date 컬럼이 존재하는가?
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    # 인덱스를 시계열로 변환
    df.index = pd.to_datetime(df.index)
    # 시작 시간을 시계열 변경 
    start = pd.to_datetime(_start)
    # 종료 시간은 타입이 문자라면 
    if type(_end) == str:
        end = pd.to_datetime(_end)
    else:
        end = _end
    # 빈 데이터 프레임을 생성 
    result = pd.DataFrame()

    for year in range(start.year, end.year):
        # 매수 시간
        buy = datetime(year=year, month=_month, day = 1 )
        # 매도 (매수의 5개월 뒤)
        sell = buy + relativedelta(months=5)

        buy_mon = buy.strftime('%Y-%m')
        sell_mon = sell.strftime('%Y-%m')
        try : 
            start_df = df.loc[buy_mon, [_col]].head(1)
            end_df = df.loc[sell_mon, [_col]].tail(1)
            result = pd.concat(
                [result, start_df, end_df], axis=0
            )
        except:
            break
    # result를 이용하여 수익율 계산
    result['rtn'] = 1
    for idx in range(1, len(result), 2):
        rtn = result.iloc[idx, ][_col] / \
                result.iloc[idx-1, ][_col]
        result.iloc[idx, -1] = rtn
    # 누적 수익율 계산
    result['acc_rtn'] = result['rtn'].cumprod()
    acc_rtn = result.iloc[-1, -1]

    return result, acc_rtn

    

In [27]:
df2 = pd.read_csv("../../csv/MSFT.csv")

In [28]:
six_month(df2)

C:\Users\moons\AppData\Local\Temp\ipykernel_4884\688789519.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9734104243660585' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn


(             Adj Close       rtn   acc_rtn
 Date                                      
 2010-11-01   21.667025  1.000000  1.000000
 2011-04-29   21.090908  0.973410  0.973410
 2011-11-01   21.420837  1.000000  0.973410
 2012-04-30   26.764505  1.249461  1.216239
 2012-11-01   25.001280  1.000000  1.216239
 2013-04-30   28.497641  1.139847  1.386326
 2013-11-01   31.021326  1.000000  1.386326
 2014-04-30   35.807358  1.154282  1.600211
 2014-11-03   42.608147  1.000000  1.600211
 2015-04-30   44.274323  1.039105  1.662787
 2015-11-02   49.098042  1.000000  1.662787
 2016-04-29   46.632641  0.949786  1.579292
 2016-11-01   56.660099  1.000000  1.579292
 2017-04-28   65.699493  1.159537  1.831248
 2017-11-01   80.711250  1.000000  1.831248
 2018-04-30   91.628487  1.135263  2.078948
 2018-11-01  104.628593  1.000000  2.078948
 2019-04-30  130.118362  1.243621  2.585424,
 2.585424226050299)

In [29]:
for i in range(1, 13, 1):
    halloween_df, halloween_rtn = six_month(df2, _month = i)
    print(f"{i}월 투자 시간시 누적 수익율 : {halloween_rtn}")

C:\Users\moons\AppData\Local\Temp\ipykernel_4884\688789519.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.7503040058806886' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn
C:\Users\moons\AppData\Local\Temp\ipykernel_4884\688789519.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9168498443317445' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn
C:\Users\moons\AppData\Local\Temp\ipykernel_4884\688789519.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.8167353995486427' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn
C:\Users\moons\AppData\Local\Temp\ipyker

1월 투자 시간시 누적 수익율 : 1.9047977289739622
2월 투자 시간시 누적 수익율 : 1.996812171930415
3월 투자 시간시 누적 수익율 : 1.8979311397394325
4월 투자 시간시 누적 수익율 : 1.8150437673363577
5월 투자 시간시 누적 수익율 : 1.9537872230675444
6월 투자 시간시 누적 수익율 : 2.3267769278149752
7월 투자 시간시 누적 수익율 : 2.742369841522477
8월 투자 시간시 누적 수익율 : 2.3189373202298746
9월 투자 시간시 누적 수익율 : 2.778991066321281
10월 투자 시간시 누적 수익율 : 2.858872297236525
11월 투자 시간시 누적 수익율 : 2.585424226050299
12월 투자 시간시 누적 수익율 : 2.4626705199785817


C:\Users\moons\AppData\Local\Temp\ipykernel_4884\688789519.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9724548658669439' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn


In [30]:
# 야후파이낸스 라이브러리 설치 
# !pip install yfinance

In [31]:
import yfinance as yf

In [32]:
# 주가 데이터 로드 
aapl_df = yf.download('AAPL', 
                      start='2023-01-01', 
                      interval="1d")

C:\Users\moons\AppData\Local\Temp\ipykernel_4884\2638380822.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  aapl_df = yf.download('AAPL',
[*********************100%***********************]  1 of 1 completed


In [35]:
six_month(
    aapl_df.stack().droplevel(level='Ticker'), 
    _start= '2023-01-01',
    _col='Close')

C:\Users\moons\AppData\Local\Temp\ipykernel_4884\688789519.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9816177599206053' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn


(Price            Close       rtn   acc_rtn
 Date                                      
 2023-11-01  172.478012  1.000000  1.000000
 2024-04-30  169.307480  0.981618  0.981618
 2024-11-01  222.129181  1.000000  0.981618
 2025-04-30  212.221710  0.955398  0.937835,
 0.9378353574540889)